In [ ]:
import run as DNN
import predict
from dirs import *
import efficiency
from data_exploration.helpers import find_file, save

import numpy as np
import numpy.typing as npt
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import keras
import timeit
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [ ]:
DO_EXPORT = False

event_name: str = "event000001001"
event = DNN.get_featured_event(event_name)
preload = True
PATH_THR = 0.85

In [ ]:
print(MODELS_ROOT)
print(SOLUTION_DIR)

In [ ]:
outrunner_model_h = DNN.get_model(preload=preload, save=DO_EXPORT, dir=SOLUTION_DIR, inname="my_model_h.h5" )

In [ ]:
used_features = event.features
used_model: keras.models.Model = outrunner_model_h # type: ignore
pred_matrix_limit: int = 10
hit_id_test = 1
hit_index_test = hit_id_test - 1

In [ ]:
x = event.hits[event.hits.hit_id == hit_id_test]
z_pos = event.hits['z'][hit_index_test] > 0

z_pos
x

In [ ]:
if z_pos:
    cand_idx = np.where(event.hits['z'] > 0)[0]
else:
    cand_idx = np.where(event.hits['z'] < 0)[0]
# TODO when z == 0

cand_idx

In [ ]:
cand_features = used_features[cand_idx]
cand_features

In [ ]:
def get_cand_idx(hit_id, hits):
    hit_index = hit_id - 1
    z_pos = hits['z'][hit_index] > 0
    if z_pos:
        cand_idx = np.where(event.hits['z'] > 0)[0]
    else:
        cand_idx = np.where(event.hits['z'] < 0)[0]    
    return cand_idx

In [ ]:
print("eliminated", len(get_cand_idx(hit_id_test, event.hits))/len(used_features))

In [ ]:
def make_predict_fast(model, features, hits: pd.DataFrame, hit_id: int, thr=0.85, batch_size: int | None = None, selector=get_cand_idx) -> npt.NDArray:
    cand_idx = selector(hit_id, hits)
    # cand_idx = np.arange(len(features))
    cand_features = features[cand_idx]

    Tx = np.zeros((len(cand_features), 10))
    # Set first five columns of Tx to be the features of the hit with hit_id
    # Shift hit_id -> hit_id - 1 because hit_id starts at 1 and index starts at 0
    hit_index = hit_id - 1
    Tx[:, :5] = np.tile(features[hit_index], (len(Tx), 1))
    # Set last five columns of Tx to be the features of all hits
    Tx[:, 5:] = cand_features

    # Make prediction
    batch_size = batch_size or round(len(Tx) / (5 * len(cand_idx)/len(features)))
    pred_small = model.predict(Tx, batch_size=batch_size)[:, 0]  # type: ignore
    pred = np.zeros(len(hits))
    pred[cand_idx] = pred_small
    
     # TTA (test time augmentation)
    """ TTA takes a similar concept but applies it during the testing or inference phase. Instead of making predictions on the original test samples alone, TTA generates multiple augmented versions of the test samples by applying various transformations or augmentations. The model then makes predictions on each augmented version, and the final prediction is obtained by aggregating the predictions from all the augmented samples. Common aggregation techniques include taking the average or the maximum probability across the augmented predictions. """

    # Take indices of prediction that have a prediction above the threshold
    idx = np.where(pred > thr)[0]

    # Filter Tx on predictions above threshold and swap first and last five columns
    # TTA
    Tx2 = np.zeros((len(idx), 10))
    # print(len(cand_idx), len(Tx), len(Tx2))
    Tx2[:, 5:] = Tx[idx, :5]
    Tx2[:, :5] = Tx[idx, 5:]

    # Predict again with swapped columns
    pred1 = model.predict(Tx2, batch_size=batch_size)[:, 0]  # type: ignore

    # Take average of predictions and swapped predictions
    pred[idx] = (pred[idx] + pred1) / 2

    return pred

make_predict_fast(used_model, used_features, event.hits, hit_id_test)
# timeit.timeit(lambda: make_predict_fast(used_model, used_features, event.hits, hit_id_test), number=100)

In [ ]:
def make_predict(
    model, features: npt.NDArray, hits: pd.DataFrame, hit_id: int, thr=0.85, batch_size: int | None = None
) -> npt.NDArray:
    """Predict probability of each pair of hits with the last hit in the path. Generates a prediction array of length len(truth) with the probability of each hit belonging to the same track as hit_id."""
    Tx = np.zeros((len(hits), 10))
    # Set first five columns of Tx to be the features of the hit with hit_id
    # Shift hit_id -> hit_id - 1 because hit_id starts at 1 and index starts at 0
    hit_index = hit_id - 1
    Tx[:, :5] = np.tile(features[hit_index], (len(Tx), 1))
    # Set last five columns of Tx to be the features of all hits
    Tx[:, 5:] = features

    # Make prediction
    batch_size = batch_size or round(len(Tx) / 5)
    pred = model.predict(Tx, batch_size=batch_size)[:, 0]  # type: ignore

    # TTA (test time augmentation)
    """ TTA takes a similar concept but applies it during the testing or inference phase. Instead of making predictions on the original test samples alone, TTA generates multiple augmented versions of the test samples by applying various transformations or augmentations. The model then makes predictions on each augmented version, and the final prediction is obtained by aggregating the predictions from all the augmented samples. Common aggregation techniques include taking the average or the maximum probability across the augmented predictions. """

    # Take indices of prediction that have a prediction above the threshold
    idx = np.where(pred > thr)[0]

    # Filter Tx on predictions above threshold and swap first and last five columns
    # TTA
    Tx2 = np.zeros((len(idx), 10))
    Tx2[:, 5:] = Tx[idx, :5]
    Tx2[:, :5] = Tx[idx, 5:]

    # Predict again with swapped columns
    pred1 = model.predict(Tx2, batch_size=batch_size)[:, 0]  # type: ignore

    # Take average of predictions and swapped predictions
    pred[idx] = (pred[idx] + pred1) / 2

    return pred
# timeit.timeit(lambda: make_predict(used_model, used_features, event.hits, hit_id_test, thr=PATH_THR, batch_size=None), number=100)
make_predict(used_model, used_features, event.hits, hit_id_test, thr=PATH_THR, batch_size=None)


In [ ]:
p_true = make_predict(used_model, used_features, event.hits, hit_id_test, thr=PATH_THR, batch_size=None)

In [ ]:
p_test = make_predict_fast(used_model, used_features, event.hits, hit_id_test, thr=PATH_THR, batch_size=None)

In [ ]:
p_true, p_test

In [ ]:
def select_seeds(hits, thr: float = 300):
    inner_idx = np.where(efficiency.add_r(hits, mode="hits")['r'] < thr)[0]
    return inner_idx
select_seeds(event.hits)

In [ ]:
pred = used_model.predict([[*used_features[0], *used_features[1]]])[:, 0]  # type: ignore
pred